In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/Users/davidsutton/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Data load

In [2]:
odf= pd.read_csv('../data/train.csv')

In [3]:
import sklearn.model_selection
index_train, index_test= sklearn.model_selection.train_test_split( odf.index, 
                                                                    test_size=0.3,random_state=1)
df= odf.loc[index_train,:]
test_df= odf.loc[index_test,:]

In [4]:
# test_df= pd.read_csv(open('../data/test.csv','rb'))

In [5]:
# yoyoma= pd.merge(
#     pd.merge( pd.DataFrame(df.groupby('ps_reg_03').target.agg(lambda x: float(len(x)) )) ,
#          pd.DataFrame(df.groupby('ps_reg_03').target.agg(lambda x: float(sum(x)) )),
#          left_index=True,right_index=True),
#     pd.DataFrame(test_df.groupby('ps_reg_03').ps_reg_03.agg(lambda x: float(len(x)) )),
#     left_index=True,right_index=True)

In [6]:
# yoyoma.tail()

In [7]:
# upscore= yoyoma[ (yoyoma.index > 2.0) & (yoyoma.target_y == 0) & (yoyoma.ps_reg_03 < 10) ].index

In [8]:
# test_ids= pd.read_csv('../data/test.csv')

In [9]:
# np.save(open('upscore_ids.bin','wb'),
#         test_ids[ map(lambda x: x in upscore,test_ids.ps_reg_03.values)].loc[:,'id'].values)

In [10]:
# Group columns by type
colnames= map(lambda x: str(x),df.columns.values)
id_col= colnames[0]
target_col= colnames[1]
cat_cols= filter(lambda x: '_cat' in x,colnames)
bin_cols= filter(lambda x: '_bin' in x,colnames)
num_cols= filter(lambda x: ('_cat' not in x) and ('_bin' not in x) and x not in [id_col,target_col] ,
                colnames)

In [11]:
def recon(reg):
    integer = int(np.round((40*reg)**2)) 
    for a in range(27):
        if (integer - a) % 27 == 0:
            A = a
#     M = (integer - A)//31
    return A

# df.loc[:,'ps_reg_04_cat'] = df.ps_reg_03.apply(recon)
# cat_cols.append('ps_reg_04_cat')

In [12]:
# np.corrcoef(df.ps_reg_02.values,df.ps_reg_03.values)

In [13]:
# df.loc[:,'c13']= pd.qcut(df.ps_car_13.values,6)
# df.groupby('c13').target.agg(lambda x: sum(x) / float(len(x)))

In [14]:
# df.loc[:,'in3_car13']= map(lambda x: str(x[0]) + '_' + str(x[1]),zip(df['ps_ind_03'].values,df['c13'].values))

In [15]:
# df.groupby('in3_car13').target.agg(lambda x: float(sum(x)) / len(x)).sort_values()

## Data exploration

In [16]:
# Cut bad fields, based on kernel discussions
bin_cols.remove('ps_ind_12_bin')
df.drop('ps_ind_12_bin',axis=1,inplace=True)

In [17]:
# tmp  = df['ps_calc_15_bin'] * 32 + df['ps_calc_16_bin'] * 16 + df['ps_calc_17_bin'] * 8
# tmp += df['ps_calc_18_bin'] * 4 + df['ps_calc_19_bin'] * 2 + df['ps_calc_20_bin'] * 1
# df.loc[:,'test_cat']= tmp

In [18]:
# Remove uninformative of noisy shadow features
shadows= ['ps_car_11_cat',       
            'ps_calc_14',        
            'ps_calc_11',         
            'ps_calc_06',           
            'ps_calc_16_bin',       
            'ps_calc_19_bin',       
            'ps_calc_20_bin',     
            'ps_calc_15_bin',      
            'ps_ind_11_bin',       
            'ps_ind_10_bin']
for s in shadows:
    if s in bin_cols:
        bin_cols.remove(s)
    elif s in num_cols:
        num_cols.remove(s)
    elif s in cat_cols:
        cat_cols.remove(s)
    df.drop(s,axis=1,inplace=True)
    
for c in df.columns:
    if 'calc' in c:
        df.drop(c,axis=1,inplace=True)
        if c in bin_cols:
            bin_cols.remove(c)
        elif c in num_cols:
            num_cols.remove(c)
        elif c in cat_cols:
            cat_cols.remove(c)

In [19]:
# # Split train and test
# import sklearn.cross_validation
# #train= range(len(df[target_col].values))
# train,test= sklearn.cross_validation.train_test_split(range(df[target_col].count()),test_size= 0.33,random_state=0)

train_df= df

In [20]:
# TO try:
# - with and without ps_reg_03 and its combo
# - add extra combos
# - convert -1s to NaNs

# Add some extra features
train_df = train_df.replace(-1, np.NaN)
d_median = train_df.median(axis=0)
d_mean = train_df.mean(axis=0)
train_df = train_df.fillna(-1)

def transform_df(indf):
    indf = pd.DataFrame(indf)
    dcol = [c for c in indf.columns if c not in ['id','target']]
    indf['ps_car_13_x_ps_reg_03'] = indf['ps_car_13'] * indf['ps_reg_03']    
#     indf['ps_reg_01_x_ps_car_02_cat'] = str(indf['ps_reg_01']) + str(indf['ps_car_02_cat'])
#     indf['ps_reg_01_x_ps_car_04_cat'] = str(indf['ps_reg_01']) + str(indf['ps_car_04_cat'])
    indf['negative_one_vals'] = np.sum((indf[dcol]==-1).values, axis=1)
    for c in dcol:
        if '_bin' not in c: #standard arithmetic
            indf[c+str('_median_range')] = (indf[c].values > d_median[c]).astype(np.int)
            indf[c+str('_mean_range')] = (indf[c].values > d_mean[c]).astype(np.int)
    return indf

train_df= transform_df(train_df)

# cat_cols.append('ps_reg_01_x_ps_car_02_cat')
# cat_cols.append('ps_reg_01_x_ps_car_04_cat')

In [21]:
# TODO: implement a class prevalence vectorizer for categoricals, to be used in processing some columns 
# and in understanding field importance

pr_prev= df.target.mean()
prior_obs= 20.
alpha= pr_prev*prior_obs
beta= prior_obs - alpha

class prevalence_vectorizer:
    def __init__(self,field_list,target_col):
        self.field_list= field_list
        self.target_col= target_col
        self.vectorizer= {}
        for f in field_list:
            self.vectorizer[f]= {}
        return None
    
    def beta_prior_ratio(self,x):
        return float(np.sum(x[self.target_col]) + alpha) / len(x[self.target_col] + beta)     
    
    def train(self,df):
        data_fields= map(lambda x: str(x),df.columns)
        for f in self.field_list:
            if f in data_fields:
                self.vectorizer[f]= {}
                grps= df.groupby(f).apply(self.beta_prior_ratio)
                for g in grps.index.values:
                    self.vectorizer[f][g]= grps[g]
            else:
                print 'Warning: field '+f+' not in train data'
        return None
    
    def parse(self,vec,field_name):
        if field_name in self.field_list:
            return map(lambda x: self.vectorizer[field_name][x] if x in self.vectorizer[field_name] else pr_prev,vec)
        else:
            print field_name + ' not in vectorizer. Available fields: '+ self.field_list
            raise Exception

## Data processing

Processing plan: 
- cut rows with missing ps_car_11 and ps_car_12
- Normalize all numerics, except: [ps_car_14 and ps_reg_03]
- For [ps_car_14 and ps_reg_03], bin and convert to class prevalence (future work: fit for the missing values?)
- Treat categoricals: dict vectorize, except for: 
     [ps_car_11_cat, ps_car_06_cat, ps_car_04_cat, ps_car_01_cat, ps_car_09_cat, ps_ind_05_cat], which we turn into class prevalence rates

TODO: better treatment of: [ps_car_14 and ps_reg_03]

In [22]:
toPrevalate= ['ps_car_06_cat','ps_car_01_cat']#,'ps_reg_04_cat']#,'ps_reg_01_x_ps_car_02_cat','ps_reg_01_x_ps_car_04_cat']
toOneHot= filter(lambda x: x not in toPrevalate,cat_cols)

In [23]:
# one hot encoder
import sklearn.preprocessing
enc= sklearn.preprocessing.OneHotEncoder(sparse= False)
X_train_onehot= enc.fit_transform( train_df.loc[:,toOneHot].as_matrix()+1.0 )

In [24]:
# Prevalence vectorizer
vectorizer= prevalence_vectorizer(toPrevalate,target_col)
vectorizer.train(train_df.loc[:,toPrevalate + [target_col] ])

X_train_prevalence= np.zeros([train_df.target.count(),len(toPrevalate)])
i=0
for c in toPrevalate:
    X_train_prevalence[:,i]= vectorizer.parse( train_df.loc[:,c].values, c ) 
    i += 1

In [25]:
# Get means and stds for the columns in training set (for non-missing values)
numerical_norms= {}
for c in num_cols:
    numerical_norms[c]= (train_df.loc[ train_df[c] >= 0, c].mean(),train_df.loc[ train_df[c] >= 0, c].std())

In [26]:
# Normalise numerical fields <- TODO: better treatment of [ps_car_14 and ps_reg_03]
X_train_numeric= np.zeros([train_df.target.count(),len(num_cols)])
i=0
for c in num_cols:
    X_train_numeric[:,i]= ( train_df[c].values - numerical_norms[c][0] ) / numerical_norms[c][1]
    i+=1

In [27]:
X_train= np.concatenate([ X_train_numeric, 
                          X_train_prevalence, 
                          X_train_onehot, 
                          train_df.loc[:,bin_cols].as_matrix() 
                        ],
                        axis=1)

X_train[X_train == -1]= np.NaN

In [28]:
np.save(open('dummy_train_matrix.bin','wb'),X_train)
np.save(open('dummy_train_labels.bin','wb'),train_df[target_col].values)

Now repeat encodings for actual test set (without fitting them)

In [29]:
# test_df= pd.read_csv(open('../data/test.csv','rb'))

In [30]:
# test_df.loc[:,'ps_reg_04_cat'] = test_df.ps_reg_03.apply(recon)

In [31]:
# tmp  = test_df['ps_calc_15_bin'] * 32 + test_df['ps_calc_16_bin'] * 16 + test_df['ps_calc_17_bin'] * 8
# tmp += test_df['ps_calc_18_bin'] * 4 + test_df['ps_calc_19_bin'] * 2 + test_df['ps_calc_20_bin'] * 1
# test_df.loc[:,'test_cat']= tmp

In [32]:
# Remove uninformative of noisy shadow features
shadows= ['ps_car_11_cat',       
            'ps_calc_14',        
            'ps_calc_11',         
            'ps_calc_06',           
            'ps_calc_16_bin',       
            'ps_calc_19_bin',       
            'ps_calc_20_bin',     
            'ps_calc_15_bin',      
            'ps_ind_11_bin',       
            'ps_ind_10_bin']
for s in shadows:
    if s in bin_cols:
        bin_cols.remove(s)
    elif s in num_cols:
        num_cols.remove(s)
    elif s in cat_cols:
        cat_cols.remove(s)
    test_df.drop(s,axis=1,inplace=True)
    
for c in test_df.columns:
    if 'calc' in c:
        test_df.drop(c,axis=1,inplace=True)
        if c in bin_cols:
            bin_cols.remove(c)
        elif c in num_cols:
            num_cols.remove(c)
        elif c in cat_cols:
            cat_cols.remove(c)

In [33]:
test_df= transform_df(test_df)

In [34]:
X_test_onehot= enc.transform( test_df.loc[:,toOneHot].as_matrix()+1.0 )
X_test_prevalence= np.zeros([test_df.id.count(),len(toPrevalate)])
i=0
for c in toPrevalate:
    X_test_prevalence[:,i]= vectorizer.parse( test_df.loc[:,c].values, c ) 
    i += 1
X_test_numeric= np.zeros([test_df.id.count(),len(num_cols)])
i=0
for c in num_cols:
    X_test_numeric[:,i]= ( test_df[c].values - numerical_norms[c][0] ) / numerical_norms[c][1]
    i+=1
X_test= np.concatenate([ X_test_numeric, 
                          X_test_prevalence, 
                          X_test_onehot, 
                          test_df.loc[:,bin_cols].as_matrix() 
                        ],
                        axis=1)
X_test[X_test == -1]= np.NaN
np.save(open('dummy_test_matrix.bin','wb'),X_test)

In [35]:
np.save(open('dummy_test_labels.bin','wb'),test_df[target_col].values)